In [1]:
# imports
import requests
from alchemyDB import *
from create_objects import *
import sqlalchemy
import time
import re
import pprint
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql.expression import func
from sqlalchemy import inspect
import pickle
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
## Run this to recreate the DB
# make_db()

In [2]:
engine = create_engine('sqlite:///boardgames.db')

Session = sessionmaker(bind=engine, autoflush=False)
session = Session()

## Connecting to the database

In [2]:
engine = create_engine('sqlite:///boardgames.db')

Session = sessionmaker(bind=engine, autoflush=False)
session = Session()

1919

In [131]:
inspector = inspect(engine)

In [132]:
print(inspector.get_table_names())

['artists', 'categories', 'games', 'games_artists', 'games_categories', 'games_mechanics', 'mechanics']


In [133]:
print([column['name'] for column in inspector.get_columns('games')])

['id', 'name', 'description', 'ratingscount', 'avgrating', 'published', 'minplayers', 'maxplayers', 'best', 'recommended', 'not_recommended', 'playingtime', 'minplaytime', 'maxplaytime', 'minage', 'suggestedage', 'language_dependence', 'designer', 'publisher']


In [5]:
# load values saved from scrape
expansions=pickle.load(open('expansions.p','rb'))
notgames=pickle.load(open('notgames.p','rb'))

In [38]:
# load game ids with more than 30 reviews
import csv

game_ids=[]
with open('2019-07-08.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            pass
        else:
            game_ids.append(int(row[0]))
        line_count+=1
    print(f'Processed {line_count} lines.')

Processed 17314 lines.


In [53]:
len(game_ids)

17313

In [99]:
# load game ids currently in DB
current_ids = [game.id for game in session.query(Game).all()]

In [100]:
len(current_ids)

17575

In [101]:
len(expansions)

22017

In [102]:
missing = list(set(game_ids)-set(current_ids))

In [103]:
# pretty sure these are expansions
len(missing)

5

In [92]:
len(list(expansions))

22017

In [123]:
[game.name for game in session.query(Game).filter(Game.id.in_(expansions)).all()]

['Blood Bowl (Third Edition)', 'Zooloretto']

## Scraping BGG

In [58]:
# run to scrape all ids with more than 30 reviews
for count, i in enumerate(game_ids):
    # make sure id is not currently in DB or marked as an expansion from a past scrape
    if (i not in expansions) & (i not in current_ids):
        address = f'https://boardgamegeek.com/xmlapi2/thing?id={i}&type=boardgame&stats=1'
        req = requests.get(address)
        if re.search(b'id=',req.content):
            # save game i to DB and save all of the ids associated with its expansions
            expansions.extend(instantiate_games(req, i))
        elif re.search(b'Rate limit exceeded.', req.content):
            # hit the server too hard, relax
            print('Zzzz')
            time.sleep(10)
            i-=1
        else:
            notgames.append(i)

    if count%100==0:
        print(f'100 more games! {get_game_collection()} collected total')
        try:
            pickle.dump(expansions, open( "expansions.p", "wb+" ) )
            pickle.dump(notgames, open("notgames.p","wb+"))
        except:
            print('No pickle for you')

[{'name': 'id', 'type': INTEGER(), 'nullable': False, 'default': None, 'autoincrement': 'auto', 'primary_key': 1}, {'name': 'name', 'type': VARCHAR(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'description', 'type': VARCHAR(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'ratingscount', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'avgrating', 'type': FLOAT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'published', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'minplayers', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'maxplayers', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'best', 'type': INTEGER(), 'nullable

In [98]:
# run to scrape the above ids that are not currently in the DB
for count, i in enumerate(missing):
    if (i not in expansions) & (i not in current_ids):
        address = f'https://boardgamegeek.com/xmlapi2/thing?id={i}&type=boardgame&stats=1'
        req = requests.get(address)
        if re.search(b'id=',req.content):
            expansions.extend(instantiate_games(req, i))
        elif re.search(b'Rate limit exceeded.', req.content):
            print('Zzzz')
            time.sleep(10)
            i-=1
        else:
            notgames.append(i)

    if count%10==0:
        print(f'10 more games! {len(missing)-count} remaining')
        try:
            pickle.dump(expansions, open( "expansions.p", "wb+" ) )
            pickle.dump(check, open("check.p","wb+"))
            pickle.dump(notgames, open("notgames.p","wb+"))
        except:
            print('No pickle for you')

10 more games! 5 remaining


In [59]:
# make sure all expansion ids found are saved
pickle.dump(expansions, open( "expansions.p", "wb+" ) )
pickle.dump(notgames, open("notgames.p","wb+"))

In [125]:
# Example result from a request
address = 'https://boardgamegeek.com/xmlapi2/thing?id=412&type=boardgame&stats=1'
req = requests.get(address)
req.content

b'<?xml version="1.0" encoding="utf-8"?><items termsofuse="https://boardgamegeek.com/xmlapi/termsofuse"><item type="boardgame" id="412">\n         <thumbnail>https://cf.geekdo-images.com/thumb/img/5NUsmxMoVQQUq4bsM-i-uFSndEg=/fit-in/200x150/pic1882905.jpg</thumbnail>\n      <image>https://cf.geekdo-images.com/original/img/KcOm1Qcx_pSFVI3VcfkA35iceEw=/0x0/pic1882905.jpg</image>\n                                     \t\t\t\t\n\t\t\t\t<name type="primary" sortindex="1" value="Yukon Company" />\n\t\t\t\n\t\t\t\t\t\t                               \t\t\t\t\n\t\t\t\t<name type="alternate" sortindex="1" value="Yukon Co." />\n\t\t\t\n\t\t\t\t\t\t               \t\t\t\t\t\t\t\t\t\t\t\t\t<description>In Yukon Company, you play a trader in Alaska who is buying goods at Dawson City and then taking them out to the prospectors at various creeks, and trying to make as much money as possible before the inevitable end of the gold rush.&amp;#10;&amp;#10;</description>\n\t\t\t\t\t\t\t\t\t\t      \t       

## Play with the data!

In [62]:
df = pd.read_sql_query("SELECT * FROM games", engine)
df.tail()

,id,name,description,ratingscount,avgrating,published,minplayers,maxplayers,best,recommended,not_recommended,playingtime,minplaytime,maxplaytime,minage,suggestedage,language_dependence,designer,publisher
17541,276894,Ticket to Ride: London,Ticket to Ride: London features the familiar g...,228,5.76794,2019,2,4,3.0,2.0,1.0,15,10,15,8,6.0,NaN,-100,-100
17542,278553,Silver,Your village has been overrun by savage werewo...,111,5.59867,2019,2,4,2.0,4.0,1.0,60,30,60,8,8.0,2.0,-100,-100
17543,278751,In Front of the Elevators,In Front of the Elevators (&#227;&#130;&#168;&...,78,5.59067,2019,2,4,3.0,2.0,1.0,40,20,40,8,NaN,NaN,-100,-100
17544,279644,Peloponnesian War,What was old is new again. I am excited to hav...,64,5.62633,2019,1,2,1.0,2.0,3.0,0,0,0,14,16.0,3.0,-100,-100
17545,280789,Pandemic: Rapid Response,Disaster has struck! Cities around the world a...,337,5.80366,2019,2,4,2.0,3.0,1.0,20,20,20,8,NaN,NaN,-100,-100


In [63]:
df.head()

,id,name,description,ratingscount,avgrating,published,minplayers,maxplayers,best,recommended,not_recommended,playingtime,minplaytime,maxplaytime,minage,suggestedage,language_dependence,designer,publisher
0,1,Die Macher,Die Macher is a game about seven sequential po...,4819,7.17548,1986,3,5,5.0,4.0,2.0,240,240,240,14,14.0,1.0,Karl-Heinz Schmiel,Hans im Glück
1,2,Dragonmaster,Dragonmaster is a trick-taking card game based...,526,5.81997,1981,3,4,4.0,3.0,1.0,30,30,30,12,2.0,2.0,"G. W. ""Jerry"" D'Arcey",E.S. Lowe
2,3,Samurai,"Part of the Knizia tile-laying trilogy, Samura...",13635,7.25468,1998,2,4,3.0,2.0,1.0,60,30,60,10,10.0,1.0,Reiner Knizia,Fantasy Flight Games
3,4,Tal der Könige,When you see the triangular box and the luxuri...,326,5.71572,1992,2,4,3.0,4.0,1.0,60,60,60,12,12.0,1.0,Christian Beierer,KOSMOS
4,5,Acquire,"In Acquire, each player strategically invests ...",17137,7.18079,1964,2,6,4.0,3.0,2.0,90,90,90,12,12.0,1.0,Sid Sackson,3M


In [72]:
df.describe()

,id,ratingscount,avgrating,published,minplayers,maxplayers,best,recommended,not_recommended,playingtime,minplaytime,maxplaytime,minage,suggestedage,language_dependence
count,17546.000000,17546.000000,17546.000000,17546.000000,17546.000000,17546.000000,14960.000000,14960.000000,14960.000000,17546.000000,17546.000000,17546.000000,17546.000000,12595.000000,12264.000000
mean,87049.953608,780.006155,5.611054,1983.349025,2.043144,5.558703,3.163837,2.497660,1.820989,96.592158,69.603442,96.592158,9.515160,9.537356,2.049821
std,85452.341274,3114.644254,0.796225,209.846419,0.678806,15.685383,1.474734,1.367836,1.701004,1069.661762,497.499859,1069.661762,3.668684,3.219822,1.223148
min,1.000000,0.000000,0.000000,-3500.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,2.000000,1.000000
25%,8045.250000,54.000000,5.509540,1998.000000,2.000000,4.000000,2.000000,1.000000,1.000000,30.000000,20.000000,30.000000,8.000000,8.000000,1.000000
50%,41762.500000,117.000000,5.553890,2009.000000,2.000000,4.000000,3.000000,2.000000,1.000000,45.000000,30.000000,45.000000,10.000000,10.000000,2.000000
75%,164560.000000,371.000000,5.698498,2015.000000,2.000000,6.000000,4.000000,3.000000,2.000000,90.000000,60.000000,90.000000,12.000000,12.000000,3.000000
max,280789.000000,86869.000000,8.599020,2020.000000,10.000000,999.000000,31.000000,22.000000,31.000000,120000.000000,60000.000000,120000.000000,25.000000,22.000000,5.000000


In [77]:
df.query('id==43231')

,id,name,description,ratingscount,avgrating,published,minplayers,maxplayers,best,recommended,not_recommended,playingtime,minplaytime,maxplaytime,minage,suggestedage,language_dependence,designer,publisher
8889,43231,Balance of Power,Game description from the publisher:&#10;&#10;...,106,5.49107,2012,2,6,6.0,5.0,1.0,120,120,120,13,10.0,1.0,-100,-100


In [95]:
df.query('name == "Beutelschneider"')['description'].iloc[0]

'Translated as Cutpurse, this is a fairly unique entry in the trick-taking genre.  Three of the suits are relatively normal, however, the cards that are worth points, in the form of gold coins, have the least value.  The special thing about this game is the trump suit, which could have the Assassin killing the King or the Adventurer seducing the Countess; the character interaction is very interesting aspect.&#10;&#10;'